In [ ]:
#The MIT License (MIT)

#Copyright (c) 2020 Juliana T.C. Marcos

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO
#THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
#CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#This code use methods of the ParticleFilter (PF) class in order to track an object in a video. The PF uses two
#measurements providers which are intermitently chosen based on the value of the ssim index of two images. The
#first image is a template of the animal to track while the second is the Region of Interest (ROI) surrounding
#the previous estimated location of the tracked animal. The measurements providers are a colour image segmentation
#technique and  one of the 8 trackers implemented in opencv. The OpenCV's trackers do not update during the colour
#image segmentation turn.

#The particles are represented in red, the PF estimate of
#the tracked animal is represented in green and the animal's bb is represented in blue. 


In [ ]:
#Import of useful librairies
import math
import numpy as np
from skimage import measure
from ParticleFilter import ParticleFilter
import cv2
import time

In [ ]:
#tracker = "BOOSTING"
tracker = "KCF"
#tracker = "CSRT"
#tracker = "MOSSE"
#tracker = "TLD"
#tracker = "MIL"
#tracker = "MEDIANFLOW"
#tracker = "GOTURN"

In [ ]:
"""Some variables initialization """

#Number of trials to average
Tot=1
#Lists for the Tot running outputs averages 
BB_avg=[]
anchor_avg=[]
xy_est_avg=[]
ssim_avg=[]
particles_avg=[]
#Video frame width and height
frame_width=1920
frame_height=1080
#number of particles
n_particles=2000
#noise in sensors' measurements
meas_noise=0
#Initialization of some important variables

font = cv2.FONT_HERSHEY_SIMPLEX
text_coord=(10,40)
t_size=0.8
t_thick=2
ssim=0
#This value was chosen according to a paper experiment
N_thresh=(2*n_particles)/30
n_resampling=0
#Lists to contain the counters for each trial
cis_l=[]
track_call_l=[]
track_ctr_l=[]
n_resampl_l=[]
#Initialize variable for shifting the anchor update between first measurements and first estimations
anchor_shift=60

In [ ]:
#Data for video cows
anchorS=(1250,350)
#This threshold helps to filter small contours
#It is however important to adapt it to the object scales in the videos
Area_thresh=0
#This is to choose the type of threshold (between cv2.THRESH_BINARY_INV 
#and THRESH_BINARY)
#for Dark or bright template
type_thr=cv2.THRESH_BINARY_INV
#type_thr=cv2.THRESH_BINARY
#std in the prediction of particles for the object's position
std=10
#Capture video where object tracking should be performed
videoIn_name="./Inputs/cows.avi"
path="./Outputs/"
videoOut_name=path+"cows-pf-ssim-colour-"+tracker+"p.avi"
template = cv2.imread('./Inputs/template2.png')
template2 = cv2.imread('./Inputs/template2_grass.png')
chg_thres=0.6
height, width =120,260
#Motion model's speed in x and y directions
v_x=0.01
v_y=0.01

In [ ]:
fyi,fxi=template2.shape[0],template2.shape[1]

In [ ]:
start_time = time.time()
for num in range(Tot):
    #List for averaging running outputs
    BB_l=[]
    anchor_l=[]
    xy_est_l=[]
    ssim_l=[]
    particles_l=[]
    #Counters
    cis=0
    track_call=0
    track_ctr=0
    it=0
    n_resampling=0
    #A single program variables initialization
    anchor=anchorS
    #Initialization of measurements variables
    x_objMeasure,y_objMeasure=anchor[0],anchor[1]
    #Initialization of some important variables
    ssim=0
    #BB variable initialization
    BB=0,0,0,0
    BB_t=0,0,0,0
    #Capture video where object tracking should be performed
    video = cv2.VideoCapture(videoIn_name)
    #Video output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_output = cv2.VideoWriter(videoOut_name,fourcc,60,(frame_width,frame_height))
    #Particles Instantiation
    particles=ParticleFilter(frame_width,frame_height, n_particles)
    
    if tracker == "BOOSTING":
        track = cv2.TrackerBoosting_create()
    elif tracker == "KCF":
        track = cv2.TrackerKCF_create()
    elif tracker == "MIL":
        track = cv2.TrackerMIL_create()
    elif tracker == "MOSSE":
        track = cv2.TrackerMOSSE_create()
    elif tracker == "TLD":
        track = cv2.TrackerTLD_create()
    elif tracker == "CSRT":
        track = cv2.TrackerCSRT_create()
    elif tracker == "MEDIANFLOW":
        track = cv2.TrackerMedianFlow_create()
    else :
        track = cv2.TrackerGOTURN_create()

    
    #This function uses opencv functions to identify the center of the animal to track
    def sensors_measurements(template,img,anchor,meas_noise=3,kernel=3):

        #Compute the mean color of the template containing the animal to track color
        meanStdTemplate=cv2.meanStdDev(template)

        #Convert the color of the frame to work on and apply the GaussianBlur function in order to remove
        #Gaussian noise, smooth image and somrtimes highlight edges
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img, (kernel, kernel), 0)

        #Binarize frame using the color mean of the animal in such a way that matching parts
        #of the frame will appear white and other parts will appear black
        ret,thresh = cv2.threshold(img,meanStdTemplate[0][1],255,type_thr)

        # find contours in the thresholded image. The if condition is used to avoid error that happens depending
        #on the python version used. The middle parameter is to only retrieve parent contours while the latter
        #is to avoid redundant points in contours.
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = cnts[0] if len(cnts) == 2 else cnts[1]

        #Compute the contours areas
        contoursAreas=[cv2.contourArea(c) for c in contours]

        #Compute the center and the BB of the contours
        contoursCenter=[]
        contoursBB=[]

        for c in range(len(contours)):
            M = cv2.moments(contours[c])
            #If the area of the current contour exists and is greater than a threshold for areas 
            if M["m00"] != 0 and contoursAreas[c]>Area_thresh:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                contoursCenter.append((c,cX,cY,contoursAreas[c]))
                contoursBB.append(cv2.boundingRect(contours[c]))

        #Compute the distance between each center and the anchor center in order to find 
        #the most suitable shape to track i.e the closest one to the anchor

        contoursAnchorDist=[math.sqrt((contoursCenter[i][1]-anchor[0])**2+\
        (contoursCenter[i][2]-anchor[1])**2) for i in range (len(contoursCenter))]

        #Save and return the coordinates of the most suitable center and its contours
        index=contoursAnchorDist.index(min(contoursAnchorDist))
        cX=contoursCenter[index][1]+np.random.standard_normal()*meas_noise
        cY=contoursCenter[index][2]+np.random.standard_normal()*meas_noise

        return cX,cY,contoursBB[index]

    #Loop through the entire video
    while (True):
        #Take the video and break it frame by frame
        _,frame=video.read()
        #Check if frames are captured
        if(_ == False ): break

        it+=1
        
        #initialization of opencv tracker
        if it==1 and _==True:
            BB_t=(int(anchor[0]-width/2),int(anchor[1]-height/2),width,height)
            track.init(frame,BB_t)
            
        #Particles prediction update
        particles.particles_update(v_x,v_y,std,frame_width,frame_height)
        particles_l.append(particles.particles.copy())
        anchor_l.append(anchor)
    
        #Take dimensions of ROI around animal in current frame for SSIM evaluation with 
        #template
        ROI_ssim=(int(anchor[0]-fxi/2),int(anchor[1]-fyi/2),fxi,fyi)
        frame_crop=frame[ROI_ssim[1]:ROI_ssim[1]+ROI_ssim[3],ROI_ssim[0]:ROI_ssim[0]+ROI_ssim[2]]
        fy,fx=frame_crop.shape[0],frame_crop.shape[1]
    
        #Here, we check if the dimensions of the ROI around the animal and the template are the same
        #to select the cis if not, because these cases correspond to the frame boundaries. Therefore, 
        #the ROI might not contain sufficient information to allow the change detection with the ssim.
        if ((fxi!=fx) or (fyi!=fy)):
            ssim=1.1
        else: 
            #The value 3 is the minimum dimension for images to compute the ssim index
            if ((fxi>3 and fyi>3)): 
                ssim=measure.compare_ssim(frame_crop,template2, \
                                              multichannel=True,win_size=3)
            else:
                ssim=0.0
        
        ssim=abs(ssim)
        ssim_l.append(ssim)
        
        if (ssim<chg_thres):
            track_call+=1
            #If a potential change is detected, call a method other than the cis to provide measurements to the PF
            cont, BB_t = track.update(frame)
            if cont==True:
                x_objMeasure,y_objMeasure=BB_t[0]+BB_t[2]/2,BB_t[1]+BB_t[3]/2
                BB=int(BB_t[0]),int(BB_t[1]),int(BB_t[2]),int(BB_t[3])
                #anchor=x_objMeasure,y_objMeasure
                
                track_ctr+=1
                print("step: ",it," x_objMeasure ",x_objMeasure," y_objMeasure ",y_objMeasure," BB_t ",BB_t)

            cv2.putText(frame,'Change detected call '+tracker+' : SSIM = '+str(ssim),text_coord,font,t_size,(255,255,255),t_thick,cv2.LINE_AA)
            
        else :
            #Measurements
            x_objMeasure,y_objMeasure,BB = sensors_measurements(template,frame,anchor,meas_noise) 
            cis+=1
            print("step: ",it," x_objMeasure ",x_objMeasure," y_objMeasure ",y_objMeasure," BB ",BB)

            cv2.putText(frame,'No Change detected, use CIS: SSIM = '+str(ssim),text_coord,font,t_size,(255,255,255),t_thick,cv2.LINE_AA)   
           
            
        #Save BB in a list and anchor in another list
        BB_l.append(BB)
        
        #Update the particles weights with the new measurement
        particles.weigth_update(x_objMeasure,y_objMeasure)

        #Estimation of object center position
        x_estimation,y_estimation=particles.position_estimation()
        print("step: ",it," x_estimation ",x_estimation," y_estimation ",y_estimation)
        
        #Save the x and y estimated in a list
        xy_est_l.append((x_estimation,y_estimation))

        #Draw the particles
        particles.draw_box_particles(frame,BB,x_estimation,y_estimation)
        #Draw the position estimation
        cv2.circle(frame,(x_estimation,y_estimation),5,[0,255,0],3)

        #Update the anchor with either the current measurements or the x and y estimates
        if it > anchor_shift:
            anchor=x_estimation,y_estimation
        else:
            anchor=x_objMeasure,y_objMeasure
        
        #Resample the particles

        if (particles.effective_particles() < N_thresh):
            n_resampling+=1
            particles.resampling()

        if (num==Tot-1):
            video_output.write(frame)
            
        #Reinitialize BB to avoid cis' bb drawing when the tracker does not track
        BB=0,0,0,0

        #if it==50:
        #    break
        
    #List for the number of resampling, cis counters,track_call and track_ctr    
    n_resampl_l.append(n_resampling)
    cis_l.append(cis)
    track_call_l.append(track_call)
    track_ctr_l.append(track_ctr)
    BB_avg.append(BB_l)
    xy_est_avg.append(xy_est_l)
    ssim_avg.append(ssim_l)
    particles_avg.append(particles_l)
    anchor_avg.append(anchor_l)


In [ ]:
prog_duration= time.time() - start_time
prog_duration

In [ ]:
prog_duration/60

In [ ]:
prog_duration/(60*Tot)

In [ ]:
sum(n_resampl_l)/Tot

In [ ]:
len(BB_l),len(xy_est_l),len(BB_avg),len(xy_est_avg)

In [ ]:
len(BB_l),len(xy_est_l),len(BB_avg),len(xy_est_avg),len(particles_avg),len(ssim_avg),\
len(anchor_avg)

In [ ]:
sum(cis_l)/Tot,sum(track_call_l)/Tot,sum(track_ctr_l)/Tot

In [ ]:
BB_l_avg=[] 
BB_avg=np.array(BB_avg)
BB_l_avg=np.sum(BB_avg,0)/Tot

In [ ]:
xy_est_l_avg=[] 
xy_est_avg=np.array(xy_est_avg)
xy_est_l_avg=np.sum(xy_est_avg,0)/Tot
xy_est_l_avg

In [ ]:
ssim_l_avg=[] 
ssim_avg=np.array(ssim_avg)
ssim_l_avg=np.sum(ssim_avg,0)/Tot

In [ ]:
particles_l_avg=[] 
particles_avg=np.array(particles_avg)
particles_l_avg=np.sum(particles_avg,0)/Tot

In [ ]:
anchor_l_avg=[] 
anchor_avg=np.array(anchor_avg)
anchor_l_avg=np.sum(anchor_avg,0)/Tot
anchor_l_avg

In [ ]:
#xy_est_l

In [ ]:
#BB_l

In [ ]:
np.save(path+'xy_data2_'+tracker+'p.npy',np.array(xy_est_l_avg))
np.save(path+'BB_data2_'+tracker+'p.npy',np.array(BB_l_avg))
np.save(path+'ssim_data2_'+tracker+'p.npy',np.array(ssim_l_avg))
np.save(path+'part_data2_'+tracker+'p.npy',np.array(particles_l_avg))
np.save(path+'anchor_data2_'+tracker+'p.npy',np.array(anchor_l_avg))